In [1]:
import eikon as ek
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ek.set_app_key('b499c15480534749a47879878c132348412d0c30')

getting the sp500 stock names from wikipedia 

In [3]:
data, _ = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [4]:
symbols = data['Symbol'].tolist()
symbols[:10]

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES']

the symbols come in the <span style="color:red">'ticker'</span> form, need to convert them to <span style="color:red">'RIC'</span> or otherwise wont be able to retrieve any data

In [18]:
def ticker_to_rics(symbols: list) -> list: 
    rics = ek.symbology.get_symbology(symbols, 
                                      from_symbol_type='ticker',
                                      to_symbol_type='RIC',
                                      bestMatch=False)
    rics = rics['RICs'].apply(func).values
    return rics.tolist()

def func(x):
    if [i for i in x[0] if i.islower()]:
        return x[1]
    return x[0]

retrieving the data for all stocks of SP500 takes about 30 seconds 

In [19]:
df, _ =  ek.get_data(instruments=ticker_to_rics(symbols), 
                     fields=['TR.ClosePrice', 'TR.PE', 'EARNINGS'])

In [20]:
df.head(10)

,Instrument,Close Price,P/E (Daily Time Series Ratio),EARNINGS
0,MMM,152.12,17.244956,8.8211
1,ABT,101.53,58.730976,1.7287
2,ABBV.K,96.04,16.994079,5.6514
3,ABMD.O,298.91,67.501468,4.4282
4,ACN,223.98,29.176599,7.6744
5,ATVI.O,81.70,40.419532,2.0213
6,ADBE.O,438.88,57.823071,7.5901
7,AMD.O,78.20,155.408494,0.5032
8,AAP,152.87,27.860143,5.4936
9,AES,15.42,35.248937,0.4375


some of the values for P/E will be NaN for the symbols with negative earnings, thats fine

In [21]:
df[df.isna().any(axis=1) == True].head(10)

,Instrument,Close Price,P/E (Daily Time Series Ratio),EARNINGS
24,MO,41.580,NaN,-0.4635
28,AAL.O,11.180,NaN,-8.1735
39,.ADI,4304.736,NaN,NaN
44,APA.O,16.010,NaN,-21.2178
60,BKR,15.330,NaN,-15.7680
71,BA,161.950,NaN,-5.0347
85,CAH,56.480,NaN,-14.0385
87,CCL,14.340,NaN,-4.0167
95,CNP,19.130,NaN,-1.0983
121,CXO,50.750,NaN,-49.3072


#### TODO address symbols with nan earnings below

In [22]:
df[df['EARNINGS'].isnull()]

,Instrument,Close Price,P/E (Daily Time Series Ratio),EARNINGS
39,.ADI,4304.736,NaN,NaN
178,ESc1,3267.750,NaN,NaN


In [29]:
df['Price at P/E = 15'] = 15*df['EARNINGS']

In [46]:
df['Diff'] = (df['Close Price'] - df['Price at P/E = 15'])/df['Close Price']

In [55]:
df.sort_values(by='Diff', ascending=True).head(10)

,Instrument,Close Price,P/E (Daily Time Series Ratio),EARNINGS,Price at P/E = 15,Diff
349,NRG,33.85,2.065898,16.3851,245.7765,-6.260753
475,VNO,35.24,2.305098,15.2922,229.3830,-5.509166
316,MGM,16.64,2.994050,5.5577,83.3655,-4.009946
464,UNM,17.16,3.655240,4.6946,70.4190,-3.103671
314,MET,37.55,3.998773,9.4000,141.0000,-2.754993
66,BRKb.N,194.30,48.692681,44.8500,672.7500,-2.462429
278,KSS,19.36,39.044066,4.3916,65.8740,-2.402583
31,AIG,31.78,6.457998,4.9189,73.7835,-1.321696
411,STX.O,44.40,6.608067,6.6514,99.7710,-1.247095
428,SYF,23.17,7.109194,3.2592,48.8880,-1.109970
